In [15]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate
import math

# Solving dy/dt = f(y,t) and Slope Fields

In [16]:
def slope_field(ax, f, t_lim, y_lim, s_len, t_step, y_step):
    ts = np.arange(t_lim[0], t_lim[1]+t_step, step=t_step)
    ys = np.arange(y_lim[0], y_lim[1]+y_step, step=y_step)

    for t in ts:
        for y in ys:
            s = f(y,t)

            fac = s_len / math.sqrt(t_step**2 + s**2*t_step**2)

            x1, x2 = t, t + fac * t_step
            y1, y2 = y, y + fac * t_step * s 

            x1, x2 = x1-(x2-x1)/2, x2-(x2-x1)/2
            y1, y2 = y1-(y2-y1)/2, y2-(y2-y1)/2

            ax.plot([x1,x2],[y1,y2], 'b', alpha=.4)

In [17]:
def ode_solve(f, ic, t_lim, y_lim, t_del=0.1):
    f_ode = lambda y,t: [f(y[0],t)]
    
    t0, y0 = ic
    ts_pos = np.arange(t0, t_lim[1]+t_del, step=t_del)
    ts_neg = np.arange(t0, t_lim[0]-t_del, step=-t_del)
    ts = np.append(ts_neg[::-1],ts_pos)

    res_pos = scipy.integrate.odeint(
        f_ode, [y0], ts_pos)

    res_neg = scipy.integrate.odeint(
        f_ode, [y0], ts_neg)

    res = np.append(res_neg[::-1], res_pos)
    
    return (ts, res)

In [20]:
def f(y, t):
    return y + t**2.0

t_step = .3
y_step = .3
t_lim = (-3,3)
y_lim = (-3,3)

s_len = 0.8*min(t_step, y_step)

In [21]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_aspect('equal')
ax.set_xlim(t_lim)
ax.set_ylim(y_lim)

slope_field(ax, f, t_lim, y_lim, s_len, t_step, y_step)

res_ode = ode_solve(f, (2,-3), t_lim, y_lim)
ax.plot(res_ode[0], res_ode[1], color="red")

plt.grid(True)
plt.show()

# dy/dt = f(y) and Phase Lines